__Problem 43__
----
Given the table below, called 'orders', write code to show the average revenue by month by channel:

In [1]:
import pandas as pd
raw_data = {'order_id': [1,2,3,4,5,6],
            'channel': ['Online', 'Online', 'In_store', 'In_store','Online','Online'],
            'date': ['2018-09-01', '2018-09-03', '2018-10-11', "2018-08-21","2018-08-13",'2018-10-29'],
            'month': ['09', '09', '10', '08','08','10'],
            'revenue': [100,125,200,80,200,100]}
df = pd.DataFrame(raw_data, columns = ['order_id', 'channel', 'date','month', 'revenue'])
df

,order_id,channel,date,month,revenue
0,1,Online,2018-09-01,09,100
1,2,Online,2018-09-03,09,125
2,3,In_store,2018-10-11,10,200
3,4,In_store,2018-08-21,08,80
4,5,Online,2018-08-13,08,200
5,6,Online,2018-10-29,10,100


Your result should return the following in a structured table:

<pre>Month | Channel | Avg. Revenue</pre>

_____
Solution

In [2]:
df_new = df.groupby(['month', 'channel'])['revenue'].mean().reset_index()
df_new.rename(columns={'revenue':'avg_revenue'})

,month,channel,avg_revenue
0,08,In_store,80.0
1,08,Online,200.0
2,09,Online,112.5
3,10,In_store,200.0
4,10,Online,100.0


Problem 44 
-----
Suppose you work for a retail company that has a database containing two tables, one called 'orders', and one called 'revenue', as shown below:

In [20]:
import sqlite3
conn = sqlite3.connect(':memory:')
crsr = conn.cursor()

def create_table_sql() -> str:
    return """
    CREATE TABLE Orders (
    order_id INTEGER,
    channel VARCHAR(1024),
    date VARCHAR(1024),
    month VARCHAR(1024)
    );
    
    CREATE TABLE Revenue (
        order_id INTEGER,
        revenue INTEGER
    );
    """

crsr.executescript(create_table_sql())
conn.commit()

In [21]:
def insert_data_sql() -> str:
    return """
    INSERT or IGNORE INTO Orders
    (order_id, channel, date, month)
    VALUES
    (1, "Online", "2018-09-01", "September"),
    (2, "Online", "2018-09-03", "September"),
    (3, "In_store", "2018-10-11", "October"),
    (4, "In_store", "2018-08-21", "August"),
    (5, "Online", "2018-08-13", "August"),
    (6, "Online", "2018-10-29", "October");
    
    INSERT or IGNORE INTO Revenue
    (order_id, revenue)
    VALUES
    (1, 100),
    (2, 125),
    (3, 200),
    (4, 80),
    (5, 200),
    (6, 100);
    """

crsr.executescript(insert_data_sql())
conn.commit()

In [50]:
def grab_order_table() -> str:
    return """
    SELECT *
    FROM Orders;
    """

crsr.execute(grab_order_table())
orders = crsr.fetchall()

pd.DataFrame(orders, columns=['order_id', 
                              'channel', 
                              'date', 
                              'month'])

,order_id,channel,date,month
0,1,Online,2018-09-01,September
1,2,Online,2018-09-03,September
2,3,In_store,2018-10-11,October
3,4,In_store,2018-08-21,August
4,5,Online,2018-08-13,August
5,6,Online,2018-10-29,October


In [51]:
def grab_revenue_table() -> str:
    return """
    SELECT *
    FROM Revenue;
    """

crsr.execute(grab_revenue_table())
revenue = crsr.fetchall()

pd.DataFrame(revenue, columns=['order_id', 'revenue'])

,order_id,revenue
0,1,100
1,2,125
2,3,200
3,4,80
4,5,200
5,6,100


In [52]:
def get_sql() -> str:
    return """
    SELECT o.channel,
           SUM(r.revenue) AS total_revenue
    FROM Orders o
    JOIN Revenue r
    ON o.order_id = r.order_id
    WHERE o.month IN ("September", "October")
    GROUP BY o.channel
    """
crsr.execute(get_sql2())
result = crsr.fetchall()
pd.DataFrame(result, columns=['channel', 'total_revenue'])

,channel,total_revenue
0,In_store,200
1,Online,325
